# MOwNiT - Rozwiązywanie równań nieliniowych

## Metoda bisekcji

Metoda bisekcji opiera się na następującym twierdzeniu:

> Jeżeli $f$ jest funkcją ciągłą w przedziale domkniętym $[a,b]$ taką, że $f(a)\cdot f(b)<0$, to równanie $f(x)=0$ ma w przedziale $(a,b)$ co najmniej jedno rozwiązanie.

Zawuważmy, że powyższe twierdzenie tak naprawdę mówi, iż w przedziale $(a,b)$ funkcja $f$ ma nieparzystą liczbę rozwiązań.

Poniższy algortym poszukuje **jednego** przybliżonego miejsca zerowego zadanej funkcji *metodą bisekcji*, czyli podziału odcinka na dwie części. Na samym początku działania algorytmu (moment startu) bierzemy pod uwagę cały odcinek $[a,b]$ i dzielimy go dokładnie na pół - obliczamy $x_0=\frac{a+b}{2}$. Jeżeli $x_0$ jest miejscem zerowy, to algorytm się kończy - szukamy jednego, dowolnego miejsca zerowego, nie wszystkich. Jeżeli $x_0$ nie jest miejscem zerowym, to mamy dwie (nie wykluczające się) możliwości:
- w przedziale $(a,x_0)$ funkcja $f$ ma nieparzystą liczbę miejsc zerowych, tzn. funkcja $f$ ma na krańcach tego przedziału przeciwne znaki
- w przedziale $(x_0,b)$ funkcja $f$ ma nieparzystą liczbę miejsc zerowych, tzn. funkcja $f$ ma na krańcach tego przedziału przeciwne znaki

Któraś z powyższych możliwości musi zachodzić (możliwe, że obydwie). Bierzemy ten przedział, na krańcach którego funkcja ma przeciwne znaki i powtarzamy całą procedurę od początku.

Oczywiście może się zdarzyć, że nasz algorytm będzie zbieżny w nieskończoności, tzn. aby punkt $x_0$ faktycznie był miejscem zerowym musielibyśmy wykonać nieskończenie wiele kroków, zatem najczęściej poszukujemy miejsca zerowego z zadaną dokładnością: w momencie, gdy długość aktualnie badanego przedziału jest już mniejsza od zadanego na początku $\varepsilon$ oraz moduł wartości funkcji w połowie przedziału $|f(x_0)|$ również jest mniejszy od $\varepsilon$, to przyjmujemy $x_0$ za dostatecznie dobre przybliżenie miejsca zerowego i kończymy algorytm.

### Algorytm

1. Start: $x_p=a$, $x_k=b$.
2. Obliczamy $x_0=\frac{x_k+x_p}{2}$.
2. Sprawdzamy, czy $x_0$ jest szukanym miejscem zerowym lub jego przylbiżeniem z wystarczającą dokładnością. Jeżeli TAK -> Koniec. Jeżeli NIE, to sprawdzamy, czy $f(x_p)f(x_0)<0$:
    - jeżeli TAK, to $x_k=x_0$ i wracamy do punktu 2.
    - jeżeli NIE, to $x_p=x_0$ i wracamy do punktu 2.

### Ćwiczenie 1.

1. Napisz funkcję ```metoda_bisekcji```, która będzie szukać miejsc zerowych zadanej funkcji $f(x)$ na zadanym przedziale $[a,b]$ z zadaną dokładnością $e$. Napisz funkcję tak, aby przy niespełnionym założeniu o przeciwnych znakach na końcach przedziału funkcja wyświetlała komunikat o błędzie.
2. Sprawdź działanie funkcji na przykładach:
    - $f(x)=x^2-1$, $x\in[0,2]$
    - $f(x)=x^3-7x+6$, $x\in[-4,4]$
    - $f(x)=\ln x$, $x\in[0.5,2]$
    - $f(x)=x^2$, $x\in[-1,1]$

In [37]:
import math
def metoda_bisekcji(f, a, b, eps):
    """
    Szuka jednego miejsca zerowego funkcji f na [a,b] metodą bisekcji
    z zadaną dokładnością eps.
    - Jeśli f(a)≈0 zwraca a, jeśli f(b)≈0 zwraca b.
    - Jeśli f(a)*f(b)>0 rzuca ValueError.
    """
    fa, fb = f(a), f(b)
    # 1) korzeń dokładnie na lewym brzegu?
    if abs(fa) < eps:
        return a
    # 2) korzeń dokładnie na prawym brzegu?
    if abs(fb) < eps:
        return b
    # 3) brak zmiany znaku → błąd
    if fa * fb > 0:
        raise ValueError(f"Brak zmiany znaku na końcach przedziału [{a}, {b}]: f(a)={fa:.3g}, f(b)={fb:.3g}")

    xp, xk = a, b
    while (xk - xp) > eps:
        x0 = (xp + xk) / 2
        f0 = f(x0)
        if abs(f0) < eps:
            return x0
        # wybieramy podprzedział ze zmianą znaku
        if fa * f0 < 0:
            xk, fb = x0, f0
        else:
            xp, fa = x0, f0
    return (xp + xk) / 2


# --- Testy ---
eps = 1e-6

# 1) f(x) = x^2 - 1, [0,2]
f1 = lambda x: x**2 - 1
print("1)", metoda_bisekcji(f1, 0, 2, eps))

# 2) f(x) = x^3 − 7x + 6, [−4,4]
f2 = lambda x: x**3 - 7*x + 6
print("2)", metoda_bisekcji(f2, -4, 4, eps))

# 3) f(x) = ln(x), [0.5,2]
f3 = lambda x: math.log(x)
print("3)", metoda_bisekcji(f3, 0.5, 2, eps))

# 4) f(x) = x^2, [−1,1]  → tu dostaniemy wyjątek
f4 = lambda x: x**2
try:
    print("4)", metoda_bisekcji(f4, -1, 1, eps))
except ValueError as e:
    print("4) Błąd:", e)

1) 1.0
2) -3.0
3) 1.0000009536743164
4) Błąd: Brak zmiany znaku na końcach przedziału [-1, 1]: f(a)=1, f(b)=1


## Metoda Newtona (metoda stycznych)

*Metoda Newtona* jest kolejnym algorytmem poszukiwania miejsc zerowych funkcji. Metoda przyjmuje następujące założenia:

> 1. Funkcja $f$ jest funkcją ciągłą i różniczkowalną na przedziale $[a,b]$.
> 2. Pierwsza i druga pochodna funkcji $f$ mają stałe znaki na przedziale $[a,b]$.
> 2. $f(a)\cdot f(b)<0$
> 2. W przedziale $[a,b]$ funkcja $f$ ma dokładnie jeden pierwiastek.

Zaczynamy od punktu będącego początkiem przedziału. Następnie wyznaczamy styczną do wykresu funkcji w puncie $(a,f(a))$ i obliczamy współrzędne $(x,0)$ jej punktu przecięcia z osią OX. Jeżeli punkt $x$ jest szukanym miejscem zerowym (lub jego odpowiednim przybliżeniem), to kończymy algorytm. W przeciwnym wypadku, dzięki stałej monotoniczności i wypukłości funkcji $f$ wiemy, że szukane miejsce zerowe znajduje się gdzieś w przedziale $(x,b]$ i powtarzamy rozumowanie.

### Algorytm

1. Start: $x_n=a$.
2. Obliczamy współrzędne $(x_{n+1},0)$ punktu przecięcia stycznej do funkcji w punkcie $(x_n,f(x_n))$ z osią OX.
3. Sprawdzamy czy $x_{n+1}$ jest miejscem zerowym lub jego odpowiednim przybliżeniem. Jeżeli TAK -> Koniec. Jeżeli NIE, to uaktualniamy $x_n=x_{n+1}$ i wracamy do punktu 2.

### Ćwiczenie 2.

1. Napisz funkcję ```metoda_Newtona```, która będzie szukać miejsc zerowych zadanej funkcji $f(x)$ na zadanym przedziale $[a,b]$ z zadaną dokładnością $e$.
2. Sprawdź działanie tej funkcji na następujących przykładach:
    - $f(x)=\ln x$, $x\in[0.5,1.5]$
    - $f(x)=x^2-1$, $x\in[0.5,2]$
3. Dlaczego algorytm nie będzie działał dla $f(x)=x^2-1$, $x\in[0,2]$?
3. Za pomocą funkcji ```metoda_Newtona``` stwórz własną funkcję, która będzie obliczać pierwiastek kwadratowy z zadanej liczby nieujemnej (bez użycia funkcji ```sqrt``` z różnych bibliotek i bez użycia potęgowania).

In [38]:
import math

def metoda_Newtona(f, df, a, b, eps=1e-6, max_iter=100):
    """
    Szuka jednego miejsca zerowego funkcji f w przedziale [a,b] metodą Newtona
    z dokładnością eps.
    - f: funkcja
    - df: pochodna f
    - a, b: końce przedziału
    - eps: tolerancja na |f(x_n)|
    - max_iter: maksymalna liczba iteracji
    Zwraca przybliżenie miejsca zerowego lub None, jeśli:
      * f(a)*f(b)>0 (brak gwarancji istnienia zera),
      * pochodna w pewnym kroku jest zerowa,
      * nie osiągnięto zbieżności w max_iter krokach.
    """
    # (1) sprawdzenie warunku na istnienie przynajmniej jednego zera
    if f(a) * f(b) > 0:
        print(f"Brak zmiany znaku na [{a}, {b}] → metoda Newtona może zawieść.")
        return None

    # (2) startujemy od środka przedziału
    x = (a + b) / 2
    for i in range(max_iter):
        fx = f(x)
        if abs(fx) < eps:
            return x
        dfx = df(x)
        if dfx == 0:
            print(f"Pochodna zerowa w x = {x:.6f} → przerwanie.")
            return None
        x = x - fx/dfx

    print("Nie osiągnięto zbieżności w zadanych iteracjach.")
    return None


# --- 2. Testy ---

eps = 1e-6

# a) f(x) = ln(x), x ∈ [0.5, 1.5]
f1  = lambda x: math.log(x)
df1 = lambda x: 1/x
root1 = metoda_Newtona(f1, df1, 0.5, 1.5, eps)
print("ln(x) on [0.5,1.5] →", root1)

# b) f(x) = x^2 - 1, x ∈ [0.5, 2]
f2  = lambda x: x**2 - 1
df2 = lambda x: 2*x
root2 = metoda_Newtona(f2, df2, 0.5, 2, eps)
print("x^2−1 on [0.5,2] →", root2)



ln(x) on [0.5,1.5] → 1.0
x^2−1 on [0.5,2] → 1.0000000464611474


Metoda Newtona może nie działać dla f(x) = x^2 - 1, x \in [0, 2], ponieważ pochodna w punkcie x = 0 (na brzegu przedziału) wynosi 0, co powoduje błąd dzielenia przez zero w pierwszym kroku iteracji, jeśli właśnie tam algorytm trafi.

In [39]:
def pierwiastek_kwadratowy(S, eps=1e-6):
    """
    Oblicza pierwiastek kwadratowy z liczby S >= 0
    metodą Newtona, bez użycia sqrt ani potęgowania.
    """
    if S < 0:
        raise ValueError("Liczba musi być nieujemna.")
    if S == 0:
        return 0.0  # pierwiastek z 0 to 0

    # f(x) = x^2 - S, df(x) = 2x
    f = lambda x: x*x - S
    df = lambda x: 2*x

    # przedział: [a, b], np. [1e-6, max(1, S)]
    a = 1e-6
    b = max(1.0, S)

    return metoda_Newtona(f, df, a, b, eps)
print(pierwiastek_kwadratowy(4))      # ≈ 2.0
print(pierwiastek_kwadratowy(2))      # ≈ 1.4142
print(pierwiastek_kwadratowy(0.25))   # ≈ 0.5

2.0000000000000626
1.4142135623746899
0.5000005


## Metoda Eulera (metoda siecznych)

Założenia metody siecznych:

> 1. Funkcja $f$ jest funkcją ciągłą na przedziale $[a,b]$.
> 2. $f(a)\cdot f(b)<0$
> 2. W przedziale $[a,b]$ funkcja $f$ ma dokładnie jeden pierwiastek.

Metoda siecznych wykorzystuje fakt, że dla odpowiednio małego przedziału $[x_1,x_2]$ sieczna przechodząca przez punkty $(x_1,f(x_1))$, $(x_2,f(x_2))$ jest wystarczająco dobrą aproksymacją funkcji $f$ na $[x_1,x_2]$. To podejście ma jedną podstawową wadę: jeżeli wyjściowy przedział $[a,b]$ jest zbyt duży, algorytm może nie zadziałać, jak powinien.

### Algorytm

1. Start: $x_1=a$, $x_2=b$.
2. Wyznaczamy punkt $(x_0,0)$ przecięcia siecznej przechodzącej przez punkty $(x_1,f(x_1))$, $(x_2,f(x_2))$ z osią OX.
3. Sprawdzamy czy $x_0$ jest miejscem zerowym lub jego odpowiednim przybliżeniem. Jeżeli TAK -> Koniec. Jeżeli NIE, to:
    - $x_1=x_2$
    - $x_2=x_0$
    - wracamy do punktu 2.

### Ćwiczenie 3.

1. Napisz funkcję ```metoda_siecznych```, która będzie szukać miejsc zerowych zadanej funkcji $f(x)$ na zadanym przedziale $[a,b]$ z zadaną dokładnością $e$.
2. Sprawdź działanie tej funkcji na następujących przykładach:
    - $f(x)=\sqrt[3]{x}$, $x\in[-1,1]$
    - $f(x)=x^3-7x+6$, $x\in[-4,-2]$
    - $f(x)=\text{arctg}\, x$, $x\in[-1,3]$
3. Dlaczego algorytm nie będzie działał dla $f(x)=x^3-7x+6$, $x\in[0,3]$?
4. Sprawdź, czy algorytm zadziała poprawnie dla $f(x)=x^2-4$, $x\in[-1,10]$.

In [40]:
import math

def metoda_siecznych(f, a, b, eps=1e-10, max_iter=100):
    """
    Szuka jednego miejsca zerowego funkcji f na [a,b] metodą siecznych (Eulera).
    Założenia:
      - f jest ciągła na [a,b],
      - f(a)*f(b) < 0 (dokładnie jedno zero w (a,b)).
    Parametry:
      f        – funkcja f(x)
      a, b     – początek i koniec przedziału
      eps      – tolerancja na |f(x_n)|
      max_iter – limit iteracji
    Zwraca:
      przybliżenie miejsca zerowego lub rzuca ValueError, gdy:
        * f(a)==0 lub f(b)==0  → od razu zwracamy a lub b,
        * f(a)*f(b)>0          → brak zmiany znaku,
        * w iteracji f(x_n)==f(x_{n-1}) → dzielenie przez zero,
        * nie osiągnięto zbieżności w max_iter krokach.
    """
    fa, fb = f(a), f(b)
    # 1) korzeń dokładnie na granicach?
    if abs(fa) < eps:
        return a
    if abs(fb) < eps:
        return b
    # 2) brak zmiany znaku
    if fa * fb > 0:
        raise ValueError(f"Brak zmiany znaku na [{a}, {b}]: f(a)={fa:.3g}, f(b)={fb:.3g}")
    x0, x1 = a, b
    f0, f1 = fa, fb

    for i in range(max_iter):
        # obliczamy punkt przecięcia siecznej z OX
        denom = (f1 - f0)
        if denom == 0:
            raise ZeroDivisionError(f"Pochodne skośne się zrównują (f1==f0) w iteracji {i}")
        x2 = x1 - f1*(x1 - x0)/denom
        f2 = f(x2)
        # warunek stopu
        if abs(f2) < eps:
            return x2
        # przesuwamy okno: x1->x0, x2->x1
        x0, f0 = x1, f1
        x1, f1 = x2, f2

    raise RuntimeError("Nie zbieżono w wyznaczonej liczbie iteracji.")


# --- 2. Testy na przykładach z ćwiczenia 3 ---

eps = 1e-6

# a) f(x) = ∛x,    x ∈ [−1,1]
f_a  = lambda x: math.copysign(abs(x)**(1/3), x)  # sześcienna pierwiastek zachowujący znak
root_a = metoda_siecznych(f_a, -1, 1, eps)
print("a) ∛x on [−1,1] →", root_a)

# b) f(x) = x^3 − 7x + 6,    x ∈ [−4,−2]
f_b  = lambda x: x**3 - 7*x + 6
root_b = metoda_siecznych(f_b, -4, -2, eps)
print("b) x^3−7x+6 on [−4,−2] →", root_b)

# c) f(x) = arctg(x),       x ∈ [−1,3]
f_c  = lambda x: math.atan(x)
root_c = metoda_siecznych(f_c, -1, 3, eps)
print("c) arctg(x) on [−1,3] →", root_c)

a) ∛x on [−1,1] → 0.0
b) x^3−7x+6 on [−4,−2] → -2.9999999997024385
c) arctg(x) on [−1,3] → 4.509058846845915e-09


Dlaczego algorytm nie zadziała dla

$f(x) = x^3 - 7x + 6,\quad x \in [0,3]$

Na przedziale $[0,3]$ mamy:  
	•	$f(0) = 6$  
	•	$f(3) = 27 - 21 + 6 = 12$  

Zatem
$$
f(0)\cdot f(3) = 6 \times 12 > 0,
$$
czyli brak zmiany znaku na końcach przedziału.

Metoda siecznych wymaga, aby początkowe punkty obejmowały korzeń funkcji (jedna wartość dodatnia, druga ujemna), w przeciwnym razie nie można rozpocząć iteracji.

In [41]:
# 4) f(x) = x^2 - 4 na [-1,10]
f4 = lambda x: x**2 - 4
root4 = metoda_siecznych(f4, -1, 10, eps)
print("4) x^2−4 on [−1,10] →", root4)

4) x^2−4 on [−1,10] → -1.9999999998922264


Algorytm wyznaczył miejsce zerowe wynoszące ok. -2, lecz nie zadziałał poprawnie, poniewaz znalezione miejsce znajduje się poza badanym przedziałem.

## Regula falsi

Założenia metody *Regula falsi* są takie same, jak założenia metody Newtona:

> 1. Funkcja $f$ jest funkcją ciągłą i różniczkowalną na przedziale $[a,b]$.
> 2. Pierwsza i druga pochodna funkcji $f$ mają stałe znaki na przedziale $[a,b]$.
> 2. $f(a)\cdot f(b)<0$
> 2. W przedziale $[a,b]$ funkcja $f$ ma dokładnie jeden pierwiastek.

Metoda jest bardzo podobna do metody siecznych z tą różnicą, że zmienia się tylko jeden z punktów, przez który przeprowadzne są kolejne sieczne. Drugi punkt jest ustalony i jest to jeden z końców odcinka.

### Algorytm

1. Start: $x_1=a$, $x_2=b$.
2. Wyznaczamy punkt $(x_0,0)$ przecięcia siecznej przechodzącej przez punkty $(x_1,f(x_1))$, $(x_2,f(x_2))$ z osią OX.
3. Sprawdzamy czy $x_0$ jest miejscem zerowym lub jego odpowiednim przybliżeniem. Jeżeli TAK -> Koniec. Jeżeli NIE, to:
    - jeżeli $f(x_1)f(x_0)<0$, to $x_2=x_0$, wartość $x_1$ się nie zmienia
    - jeżeli $f(x_2)f(x_0)<0$, to $x_1=x_0$, wartość $x_2$ się nie zmienia
    - wracamy do punktu 2.
    
Dzięki założeniom funkcja $f$ jest monotoniczna w $[a,b]$, ma również określoną wypukłość, zatem w punkcie 3 algorytmu zawsze będzie wybierana ta sama opcja i przez cały algorytm jedna z wartości $x_1=a$ albo $x_2=b$ pozostanie bez zmian. Można na samym początku algorytmu ustalić, która z tych wartości pozostanie stała i zrezygnować z dwóch instrukcji warunkowych w pętli, co znacznie przyspiesza obliczenia, jednak wtedy algorytm jest mniej uniwersalny.

### Ćwiczenie 4.

1. Napisz funkcję ```regula_falsi```, która będzie szukać miejsc zerowych zadanej funkcji $f(x)$ na zadanym przedziale $[a,b]$ z zadaną dokładnością $e$.
2. Sprawdź działanie tej funkcji na przykładach:
    - $f(x)=e^x-1$, $x\in[-1,1]$
    - $f(x)=x^2-1\, x$, $x\in[0,2]$
4. Sprawdź, czy algorytm zadziała poprawnie w przypadku funkcji:
    - $f(x)=x^2-4$, $x\in[-1,10]$ (funkcja nie jest monotoniczna, ale ma stałą wypukłość)
    - $f(x)=\sqrt[3]{x}$, $x\in[-3,3]$ (funkcja ma punkt przegięcia, ale jest monotoniczna)
    - $f(x)=\sin x$, $x\in[-2,2]$ (funkcja ma punkt przegięcia i nie jest monotoniczna)

In [42]:
import math

def regula_falsi(f, a, b, tol=1e-6, max_iter=100):
    """
    Szuka miejsca zerowego funkcji f na przedziale [a, b] z dokładnością tol.
    Zwraca przybliżenie pierwiastka oraz liczbę iteracji.
    """
    fa, fb = f(a), f(b)
    if fa * fb >= 0:
        raise ValueError("f(a) i f(b) muszą mieć przeciwne znaki")
    for i in range(1, max_iter + 1):
        # oblicz przecięcie siecznej z osią OX
        x0 = b - fb * (b - a) / (fb - fa)
        fx0 = f(x0)
        # sprawdź kryterium zbieżności
        if abs(fx0) < tol:
            return x0, i
        # wybierz podprzedział, w którym jest pierwiastek
        if fa * fx0 < 0:
            b, fb = x0, fx0
        else:
            a, fa = x0, fx0
    return x0, max_iter

if __name__ == "__main__":
    tests = [
        ("e^x - 1",  lambda x: math.exp(x) - 1,                 -1, 1),
        ("x^2 - 1",  lambda x: x**2 - 1,                       0, 2),
        ("x^2 - 4",  lambda x: x**2 - 4,                      -1, 10),
        ("∛x",       lambda x: math.copysign(abs(x)**(1/3), x), -3, 3),
        ("sin(x)",   lambda x: math.sin(x),                   -2, 2),
    ]

    tol = 1e-8
    for name, f, a, b in tests:
        print(f"\nTest: {name} on [{a}, {b}]")
        try:
            root, iters = regula_falsi(f, a, b, tol=tol, max_iter=100)
            residuum = abs(f(root))
            status = "OK" if residuum < tol else "NIEZBIEŻNE"
            print(f"  Pierwiastek ≈ {root}")
            print(f"  f(root) = {f(root)} (|f(root)| = {residuum})")
            print(f"  Iteracje = {iters}")
            print(f"  Status: {status}")
        except Exception as e:
            print(f"  ERROR: {e}")


Test: e^x - 1 on [-1, 1]
  Pierwiastek ≈ -5.607373010363403e-09
  f(root) = -5.607373010363403e-09 (|f(root)| = 5.607373010363403e-09)
  Iteracje = 22
  Status: OK

Test: x^2 - 1 on [0, 2]
  Pierwiastek ≈ 0.9999999982792167
  f(root) = -3.4415665872700174e-09 (|f(root)| = 3.4415665872700174e-09)
  Iteracje = 19
  Status: OK

Test: x^2 - 4 on [-1, 10]
  Pierwiastek ≈ 1.9999999975216536
  f(root) = -9.913385667914554e-09 (|f(root)| = 9.913385667914554e-09)
  Iteracje = 55
  Status: OK

Test: ∛x on [-3, 3]
  Pierwiastek ≈ -9.693585472049334e-25
  f(root) = -9.89680047341996e-09 (|f(root)| = 9.89680047341996e-09)
  Iteracje = 25
  Status: OK

Test: sin(x) on [-2, 2]
  Pierwiastek ≈ 0.0
  f(root) = 0.0 (|f(root)| = 0.0)
  Iteracje = 1
  Status: OK


### Dlaczego metoda falsi działa w przypadkach funkcji z punktami przegięcia lub bez monotoniczności?

Metoda falsi (regula falsi) działa poprawnie, gdy spełnione są dwa warunki:

1. Funkcja jest ciągła na przedziale $[a, b]$  
2. Występuje zmiana znaku: $f(a) \cdot f(b) < 0$

Nie jest wymagane, aby funkcja była monotoniczna czy wypukła.

#### Przykłady:

- $f(x) = x^2 - 4$ na $[-1, 10]$: funkcja nie jest monotoniczna, ale ciągła i zachodzi zmiana znaku → metoda działa.
- $f(x) = \sqrt[3]{x}$ na $[-3, 3]$: funkcja ma punkt przegięcia w $x = 0$, ale jest ciągła i monotoniczna → metoda działa.
- $f(x) = \sin(x)$ na $[-2, 2]$: funkcja nie jest monotoniczna i ma punkt przegięcia, ale warunki metody są spełnione → działa poprawnie.

### Ćwiczenie 5.

W pakiecie `scipy.optimize` znajdują się dwie funkcje do znajdowania pierwiastków: `fsolve()` oraz `root()`. Wykorzystując dokumentację biblioteki `scipy` za pomocą odpowiednio dobranej funkcji rozwiąż te przykłady z ćwiczeń 1-4, których nie udało się rozwiązać podstawowymi algorytmami.

In [56]:
from scipy.optimize import fsolve
from scipy.optimize import root

f1 = lambda x: x**2
result = root(f1, x0=1.0)
print("root dla x^2:", result.x[0])

f2 = lambda x: x**2 - 1
x0 = 0.5
wynik_fsolve = fsolve(f2, x0)
print("fsolve dla x^2 - 1:", wynik_fsolve[0])

f3 = lambda x: x**3 - 7*x + 6
print("Pierwiastek dla x^3 - 7x + 6:", fsolve(f3, x0=2.5)[0])



root dla x^2: 5.681053833357605e-84
fsolve dla x^2 - 1: 1.0000000000000002
Pierwiastek dla x^3 - 7x + 6: 2.000000000000014
